### Importations

In [ ]:
import orbit_modelling_library as orbit
import math as m
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from mpl_toolkits.mplot3d import Axes3D
rc('text', usetex = True)

# Fonction $\chi^2$

In [ ]:
def chi_2(observation, model, delta_radar, delta_accelerometer, delta_initial_position, delta_initial_speed):
    if len(observation.t) != len(model.t):
        print("Error : the observation and the model are not sampled at the same times")
        return
    for i in range(len(observation.t)):
        if observation.t[i] != model.t[i]:
            print("Error : the observation and the model are not sampled at the same times")
            return
    
    chi_2 = 0
    delta_position = delta_initial_position
    delta_speed = delta_initial_speed
    T = observation.t
    t0 = T[0]
    time_step = T[1] - T[0]
    
    for i in range(len(T)):
        X_obs, X_mod = observation.X[i], model.X[i]
        Y_obs, Y_mod = observation.Y[i], model.Y[i]
        Z_obs, Z_mod = observation.Z[i], model.Z[i]
        
        delta_position += time_step * delta_speed
        delta_speed += time_step * delta_accelerometer
        
        chi_2 += ((X_obs - X_mod)**2 + (Y_obs - Y_mod)**2 + (Z_obs - Z_mod)**2) / (delta_radar**2 + delta_position**2)
    return chi_2 

### Test de la fonction $\chi^{2}$

In [ ]:
observation = orbit.Orbit_model("2050-01-01T00:00:00", "2050-02-01T00:00:00", 10**6, "Yukawa_1_month", type = "Yukawa")
model = orbit.Orbit_model("2050-01-01T00:00:00", "2050-02-01T00:00:00", 10**6, "all_forces_1_month", type = "Newton")
observation.load()
model.load()

delta_radar = 1
delta_accelerometer = 10**(-12)
delta_initial_position = delta_radar
delta_initial_speed = 0

print(chi_2(observation, model, delta_radar, delta_accelerometer, delta_initial_position, delta_initial_speed))

In [ ]:
m.sqrt(delta_radar + 0.5*delta_accelerometer*(86400*30)**2)

Pour rappel, on avait obtenu une distance finale entre ces 2 trajectoires de 15m, soit environ 7 $\sigma$ d'après le calcul de la cellule précédente. Comme on trouve une certitude à 1000 $\sigma$, on pense qu'il y a une erreur quelque part...

# Data generation

In [ ]:
def observations_data_generation(time_isot_start, time_isot_end, n_points_t, 
                                      lambda_start, lambda_end, n_points_lambda, 
                                      alpha_start, alpha_end, n_points_alpha, 
                                      target_files, 
                                      Bodies = [i + 1 for i in range(10)], radiation = False, solar_wind = False, 
                                      reflectivity_sat = 0.5, radius_sat = 0.63, mass_sat = 100, plasma_speed = 450000):
    Lambda = [lambda_start + (lambda_end - lambda_start) * i / (n_points_lambda - 1) for i in range(n_points_lambda)]
    Alpha = [alpha_start + (alpha_end - alpha_start) * i / (n_points_alpha - 1) for i in range(n_points_alpha)]
    
    N = n_points_lambda*n_points_alpha
    print(N)
    
    i = 0
    print(100 * i / (N - 1))
    for lambdaa in Lambda:
        for alpha in Alpha:
            i += 1
            target_file = target_files + "_lambda_=_" + str(lambdaa) + "_alpha_=_" + str(alpha)
            observation = orbit.Orbit_model(time_isot_start, time_isot_end, n_points_t, target_file, Bodies, "Yukawa", alpha, lambdaa, radiation, solar_wind, reflectivity_sat, radius_sat, mass_sat, plasma_speed)
            observation.compute()
            print(100 * i / N) # gives the pourcentage of the computation that has already been done

In [ ]:
def model_data_generation(time_isot_start, time_isot_end, n_points_t, target_files, 
                          Bodies = [i + 1 for i in range(10)], type = "Yukawa", alpha = 10**(-4), lambdaa = 10**12, 
                          radiation = False, solar_wind = False, 
                          reflectivity_sat = 0.5, radius_sat = 0.63, mass_sat = 100, plasma_speed = 450000):
    
    target_file = target_files + "_model"
    model = orbit.Orbit_model(time_isot_start, time_isot_end, n_points_t, target_file, Bodies, type, alpha, lambdaa, radiation, solar_wind, reflectivity_sat, radius_sat, mass_sat, plasma_speed)
    model.compute()

### Test du bloc data generation

In [ ]:
model_data_generation("2050-01-01T00:00:00", "2050-02-01T00:00:00", 10**4, "Test_data_generation")
print("Ok_model")
observations_data_generation("2050-01-01T00:00:00", "2050-02-01T00:00:00", 10**4, 
                            10**(-12), 10**(-11), 5, 
                            10**(-5), 10**(-4), 5, 
                            "Test_data_generation")
print("Ok_observation")

Le code compile, mais comme on a pas demandé assez de points en temps il y a des valeurs absurdes, ce qui donne des résultats inexploitables pour tester la correction

# Data usage

In [ ]:
def grid(time_isot_start, time_isot_end, n_points_t, target_files,
         lambda_start, lambda_end, n_points_lambda, 
         alpha_start, alpha_end, n_points_alpha, 
         delta_radar, delta_accelerometer, delta_initial_position, delta_initial_speed, 
         Bodies = [i + 1 for i in range(10)], type_model = "Yukawa", alpha_model = 10**(-4), lambdaa_model = 10**12,
         radiation = False, solar_wind = False, 
         reflectivity_sat = 0.5, radius_sat = 0.63, mass_sat = 100, plasma_speed = 450000):
    Lambda = [lambda_start + (lambda_end - lambda_start) * i / (n_points_lambda - 1) for i in range(n_points_lambda)]
    Alpha = [alpha_start + (alpha_end - alpha_start) * i / (n_points_alpha - 1) for i in range(n_points_alpha)]
    
    target_file = target_files + "_model"
    model = orbit.Orbit_model(time_isot_start, time_isot_end, n_points_t, target_file, Bodies, type_model, alpha_model, lambdaa_model, radiation, solar_wind, reflectivity_sat, radius_sat, mass_sat, plasma_speed)
    model.load()
    
    Z = [[0 for j in range(n_points_alpha)] for i in range(n_points_lambda)]
    for i in range(len(Lambda)):
        lambdaa = Lambda[i]
        for j in range(len(Alpha)):
            alpha = Alpha[j]
            target_file = target_files + "_lambda_=_" + str(lambdaa) + "_alpha_=_" + str(alpha)
            observation = orbit.Orbit_model(time_isot_start, time_isot_end, n_points_t, target_file, Bodies, "Yukawa", alpha, lambdaa, radiation, solar_wind, reflectivity_sat, radius_sat, mass_sat, plasma_speed)
            observation.load()
            Z[i][j] = chi_2(observation, model, delta_radar, delta_accelerometer, delta_initial_position, delta_initial_speed)
            
    return Z

In [ ]:
def drawing(lambda_start, lambda_end, n_points_lambda, 
            alpha_start, alpha_end, n_points_alpha, 
            grid_alpha_lambda):
    X = [lambda_start + (lambda_end - lambda_start) * i / (n_points_lambda - 1) for i in range(n_points_lambda)]
    Y = [alpha_start + (alpha_end - alpha_start) * i / (n_points_alpha - 1) for i in range(n_points_alpha)]
    Z = np.asarray(grid_alpha_lambda)

    fig = plt.figure(1)
    X, Y = np.meshgrid(X, Y)
    ax = plt.axes(projection = '3d')
    ax.plot_surface(X, Y, Z, rstride = 1, cstride = 1, cmap = 'viridis', edgecolor = 'none')

### Test du bloc data_usage

In [ ]:
delta_radar = 1
delta_accelerometer = 10**(-12)
delta_initial_position = delta_radar
delta_initial_speed = 0

Z = grid("2050-01-01T00:00:00", "2050-02-01T00:00:00", 10**3, "Test_data_generation", 
         10**(-12), 10**(-11), 5, 
         10**(-5), 10**(-4), 5, 
         delta_radar, delta_accelerometer, delta_initial_position, delta_initial_speed)

drawing(10**(-12), 10**(-11), 5, 
         10**(-5), 10**(-4), 5, 
        Z)

Le code compile et trace la courbe 3D, ce qui est bon signe